In [1]:
import xarray as xr
import gcsfs
import glob
import numpy as np

In [2]:
from dask_gateway import Gateway
from dask.distributed import Client
gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt(minimum=1, maximum=20)
client = Client(cluster)
cluster

In [3]:
zstore = 'gs://pangeo-nasa-ccmp/zarr'
fs = gcsfs.GCSFileSystem(project='pangeo-181919',requester_pays=True)
files = fs.ls('pangeo-nasa-ccmp',project='pangeo-181919')
mapper = fs.get_mapper(zstore)
ds = xr.open_zarr(mapper)#, consolidated=True)
ds['wspd']=np.sqrt(ds.uwnd**2+ds.vwnd**2)

In [4]:
ds

,Array,Chunk
Bytes,174.09 GB,226.08 MB
Shape,"(48128, 628, 1440)","(2000, 157, 180)"
Count,801 Tasks,800 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,174.09 GB,226.08 MB
Shape,"(48128, 628, 1440)","(2000, 157, 180)"
Count,801 Tasks,800 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [5]:
%%time
ds_climatology = ds.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)
ds_anomaly = ds.groupby('time.dayofyear')-ds_climatology
ds_anomaly_monthly = ds_anomaly.resample(time='1MS').mean()

#plot the data
#ds_anomaly.mean({'latitude','longitude'}).plot()
#ds_anomaly_monthly.plot()
#plt.axhline(linewidth=2,color='k')

CPU times: user 1min 22s, sys: 6.61 s, total: 1min 28s
Wall time: 1min 28s


In [6]:
%%time
ts = ds_anomaly.mean({'latitude','longitude'})
ts_mon = ds_anomaly_monthly.mean({'latitude','longitude'})
#ts.plot()
#ts_mon.plot()
ts

CPU times: user 12.3 s, sys: 1.44 s, total: 13.7 s
Wall time: 13.7 s


,Array,Chunk
Bytes,192.51 kB,16 B
Shape,"(48128,)","(4,)"
Count,2180257 Tasks,12032 Chunks
Type,float32,numpy.ndarray


In [ ]:
%%time
ts.wspd.plot()
ts_mon.wspd.plot()

In [ ]:
ds.wspd[-10,:,:].plot()

In [ ]:
ds.wspd[10,:,:].plot()

In [ ]:
ds.wspd[:,800,400].plot()